In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\joszp\.wdm\drivers\chromedriver\win32\92.0.4515.107]


In [4]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# Create variables for connection to website
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [6]:
# Find website main elements to parse
slide_elem.find('div', class_='content_title')

<div class="content_title">Global Storms on Mars Launch Dust Towers Into the Sky</div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Global Storms on Mars Launch Dust Towers Into the Sky'

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.'

### Featured Images

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [14]:
# Read HTML data and create a DataFrame. Format index and columns
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# Convert DataFrame to HTML
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [16]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

In [23]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

In [18]:
# Retrieve html from webpage and parse with BeautifulSoup
html = browser.html
soup = soup(html, "html.parser")

In [19]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.

# Create empty lists to place iteration values
title_list = []
photos_list = []

# Find numbers of articles to iterate
number_articles = len(browser.links.find_by_partial_text('Enhanced'))

for x in range(0, number_articles):
    # Navigate browser to each article and append titles to a list
    browser.links.find_by_partial_text('Enhanced')[x].click()
    temp_title = browser.find_by_tag('h2')[0]
    title_list.append(temp_title.value)
    
    # Navigate browser to .jpg image and append link to a list
    browser.links.find_by_partial_text('Sample').click()
    temp_html = browser.url
    photos_list.append(temp_html)
    
    # Close opened tab
    browser.windows[1].close()
    
    # Navigate browser to main page
    browser.back()
    
    continue
    

In [20]:
# Review list of photos url
photos_list

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [21]:
# Review list of titles
title_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [25]:
# Convert lists into a dictionary 
temp_dict = dict(zip(title_list, photos_list))

# Append dictionary to hemisphere_image_urls list
hemisphere_image_urls.append(temp_dict)

# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'Cerberus Hemisphere Enhanced': 'https://marshemispheres.com/cerberus.html',
  'Schiaparelli Hemisphere Enhanced': 'https://marshemispheres.com/schiaparelli.html',
  'Syrtis Major Hemisphere Enhanced': 'https://marshemispheres.com/syrtis.html',
  'Valles Marineris Hemisphere Enhanced': 'https://marshemispheres.com/valles.html'},
 {'Cerberus Hemisphere Enhanced': 'https://marshemispheres.com/cerberus.html',
  'Schiaparelli Hemisphere Enhanced': 'https://marshemispheres.com/schiaparelli.html',
  'Syrtis Major Hemisphere Enhanced': 'https://marshemispheres.com/syrtis.html',
  'Valles Marineris Hemisphere Enhanced': 'https://marshemispheres.com/valles.html'}]

In [26]:
# 5. Quit the browser
browser.quit()